In [1]:
import os

In [2]:
%pwd

'd:\\MLpipeline\\mlpipeline-example\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\MLpipeline\\mlpipeline-example'

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [12]:
# import 

from src.mlProject.constants import *
# whatever things are inside constants import everything

from src.mlProject.utils.common import read_yaml,create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
            
        )

        return model_trainer_config

In [14]:
import pandas as pd
import os
from src.mlProject import logger
from sklearn.linear_model import ElasticNet
import joblib

In [15]:

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train_model(self):
        # Load the training data
        train_data = pd.read_csv(self.config.train_data_path)

        # Separate features and target
        X_train = train_data.drop(columns=[self.config.target_column])
        y_train = train_data[self.config.target_column]

        # Identify categorical and numerical columns
        categorical_columns = X_train.select_dtypes(include=['object']).columns
        numerical_columns = X_train.select_dtypes(include=['number']).columns

        # Preprocessing for categorical and numerical features
        preprocessor = ColumnTransformer(
            transformers=[
                ('num', StandardScaler(), numerical_columns),  # Scale numerical features
                ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_columns)  # Encode categorical features
            ]
        )

        # Create a pipeline with preprocessing and model
        model = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('classifier', LogisticRegression(
                C=self.config.alpha, 
                l1_ratio=self.config.l1_ratio, 
                solver='saga', 
                penalty='elasticnet'
            ))
        ])

        # Train the model
        model.fit(X_train, y_train)

        # Save the trained model
        os.makedirs(self.config.root_dir, exist_ok=True)
        joblib.dump(model, os.path.join(self.config.root_dir, self.config.model_name))
        print(f"Model saved at: {os.path.join(self.config.root_dir, self.config.model_name)}")


In [16]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-12-23 14:39:46,096: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-23 14:39:46,101: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-23 14:39:46,112: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-23 14:39:46,115: INFO: common: created directory at: artifacts]
[2024-12-23 14:39:46,119: INFO: common: created directory at: artifacts/model_trainer]
